In [87]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import matplotlib.pyplot as plt

import folium
import rasterio
import rasterio.mask

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from geopy.distance import geodesic
import folium
from sklearn.tree import DecisionTreeClassifier

In [88]:
incoming_weather_size_draught = pd.read_csv('Data/Input_ML/incoming_weather_size_draught.csv')
incoming_weather_size = pd.read_csv('Data/Input_ML/incoming_weather_size.csv')
incoming_weather = pd.read_csv('Data/Input_ML/incoming_weather.csv')

In [89]:
# Assuming 'column_name' is the name of the column containing strings in your DataFrame
label_encoder = LabelEncoder()
incoming_weather_size_draught['Direction'] = label_encoder.fit_transform(incoming_weather_size_draught['Direction'])
incoming_weather_size_draught['Wind_speed'] = label_encoder.fit_transform(incoming_weather_size_draught['Wind_speed'])
incoming_weather_size_draught['Type'] = label_encoder.fit_transform(incoming_weather_size_draught['Type'])
#incoming_weather_size_draught['to_haven'] = label_encoder.fit_transform(incoming_weather_size_draught['to_haven'])
incoming_weather_size_draught

,Unnamed: 0,area_ID,geometry,index_right,from,from_rounded,from_location,to_haven,trip_ID,vessel_mmsi,Datetime,Direction,Wind_speed,device_mmsi_x,Lenght,Width,Type,device_mmsi_y,Navigation_draught,DateTime
0,0,1584,"POLYGON ((4.141337398344955 51.96305993744309,...",8,2022-05-31 23:34:58+00:00,2022-06-01 00:00:00+00:00,"[4.14163, 51.96557]",3PET,4,538005865,2022-06-01 00:00:00+00:00,6,2,538005865.0,159.0,27.0,2,538005865,8.3,2022-06-01 00:00:00+00:00
1,1,2573,POLYGON ((4.306675663908258 51.893109132781674...,10632,2022-09-08 07:01:29+00:00,2022-09-08 07:00:00+00:00,"[4.30278, 51.89511]",1PET,6054,538005865,2022-09-08 07:00:00+00:00,5,3,538005865.0,159.0,27.0,2,538005865,9.6,2022-09-08 07:00:00+00:00
2,2,1820,MULTIPOLYGON (((4.185851546765845 51.937623281...,10997,2022-09-11 04:34:51+00:00,2022-09-11 05:00:00+00:00,"[4.17956, 51.93872]",CKBRUG,6258,538005865,2022-09-11 05:00:00+00:00,0,2,538005865.0,159.0,27.0,2,538005865,9.0,2022-09-11 05:00:00+00:00
3,3,2335,"POLYGON ((4.268520679547496 51.90582746090193,...",20123,2022-11-24 02:30:45+00:00,2022-11-24 03:00:00+00:00,"[4.26256, 51.90621]",3PET,11529,538005865,2022-11-24 03:00:00+00:00,7,1,538005865.0,159.0,27.0,2,538005865,7.6,2022-11-24 03:00:00+00:00
4,4,2453,MULTIPOLYGON (((4.284689496769173 51.893109132...,14458,2022-10-10 08:13:47+00:00,2022-10-10 08:00:00+00:00,"[4.28256, 51.89866]",TORONT,8267,538005077,2022-10-10 08:00:00+00:00,5,3,538005077.0,158.0,27.0,2,538005077,10.3,2022-10-10 08:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4042,4042,1661,POLYGON ((4.154055726465209 51.950341609322834...,23823,2022-12-25 00:56:07+00:00,2022-12-25 01:00:00+00:00,"[4.15802, 51.95009]",7PET,13637,538003962,2022-12-25 01:00:00+00:00,5,3,538003962.0,183.0,32.0,2,538003962,11.3,2022-12-25 01:00:00+00:00
4043,4043,1027,"POLYGON ((4.052309101503177 51.98849659368361,...",23850,2022-12-25 08:17:19+00:00,2022-12-25 08:00:00+00:00,"[4.05757, 51.98543]",SCHEUR,13651,477345300,2022-12-25 08:00:00+00:00,5,3,477345300.0,225.0,32.0,0,477345300,14.0,2022-12-25 08:00:00+00:00
4044,4044,1265,"POLYGON ((4.090464085863939 51.96941910150322,...",23857,2022-12-25 10:19:15+00:00,2022-12-25 10:00:00+00:00,"[4.09324, 51.97466]",5PET,13655,538008798,2022-12-25 10:00:00+00:00,5,3,538008798.0,245.0,42.0,2,538008798,15.3,2022-12-25 10:00:00+00:00
4045,4045,2573,POLYGON ((4.306675663908258 51.893109132781674...,23867,2022-12-25 12:25:08+00:00,2022-12-25 12:00:00+00:00,"[4.30134, 51.89561]",2PET,13660,403579000,2022-12-25 12:00:00+00:00,5,3,403579000.0,183.0,32.0,2,403579000,11.4,2022-12-25 12:00:00+00:00


In [90]:
leaving_weather_size_draught = pd.read_csv('Data/Input_ML/leaving_weather_size_draught.csv')
leaving_weather_size = pd.read_csv('Data/Input_ML/leaving_weather_size.csv')
leaving_weather = pd.read_csv('Data/Input_ML/leaving_weather.csv')

In [91]:
# Assuming 'column_name' is the name of the column containing strings in your DataFrame
label_encoder = LabelEncoder()
leaving_weather_size_draught['Direction'] = label_encoder.fit_transform(leaving_weather_size_draught['Direction'])
leaving_weather_size_draught['Wind_speed'] = label_encoder.fit_transform(leaving_weather_size_draught['Wind_speed'])
leaving_weather_size_draught['Type'] = label_encoder.fit_transform(leaving_weather_size_draught['Type'])
#leaving_weather_size_draught['from_haven'] = label_encoder.fit_transform(leaving_weather_size_draught['from_haven'])

In [92]:
leaving_top = leaving_weather_size_draught['from_haven'].value_counts()
incoming_top = incoming_weather_size_draught['to_haven'].value_counts()
print(leaving_top[0:2])
print(incoming_top[0:2])

from_haven
EURO      451
AMALIA    309
Name: count, dtype: int64
to_haven
EURO    480
7PET    377
Name: count, dtype: int64


In [93]:
top_leaving = 'EURO'
top_incoming = 'EURO'

harbor_leaving = leaving_weather_size_draught[leaving_weather_size_draught['from_haven']== top_leaving]
harbor_incoming = incoming_weather_size_draught[incoming_weather_size_draught['to_haven']== top_incoming]


In [94]:
columns_leaving = [
    #'from_haven',
    'Direction',
    'Wind_speed',
    'Lenght'	,
    'Width'	,
    'Type'
]

columns_incoming = [
    #'to_haven',
    'Direction',
    'Wind_speed',
    'Lenght'	,
    'Width'	,
    'Type'
]


In [95]:
X_from_lat_train, X_from_lat_test, y_from_lat_train, y_from_lat_test = train_test_split(harbor_leaving[columns_leaving], harbor_leaving['area_ID'], test_size=0.3, random_state=42)
#X_from_lat_train, X_from_lat_test, y_from_lat_train, y_from_lat_test = train_test_split(leaving_weather_size_draught[columns_leaving], leaving_weather_size_draught['area_ID'], test_size=0.3, random_state=42)

In [96]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Initialize the Decision Tree Classifier
decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# Train the classifier using the training data
decision_tree_classifier.fit(X_from_lat_train, y_from_lat_train)

# Make predictions on the test data
predictions = decision_tree_classifier.predict(X_from_lat_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_from_lat_test, predictions)

print("Accuracy of the Decision Tree Classifier: {:.2f}%".format(accuracy * 100))


Accuracy of the Decision Tree Classifier: 18.38%
